# DATA CLEANING

#### MODULES IMPORT

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math as math

#### RAW DATA IMPORT

In [5]:
df_2015 = pd.read_csv('./raw/marathon_results_2015.csv')
df_2016 = pd.read_csv('./raw/marathon_results_2016.csv')
df_2017 = pd.read_csv('./raw/marathon_results_2017.csv')

#### ADDING YEAR COLUMNS TO EACH DATASET BEFORE COMBINING THEM TO KNOW YEAR FROM EVERY RESULT

In [7]:
df_2015['Year'] = 2015
df_2016['Year'] = 2016
df_2017['Year'] = 2017

#### COMBINING IMPORTED DATASETS INTO ONE

In [9]:
df = pd.concat([df_2015, df_2016, df_2017])

#### SELECTING COLUMNS

In [11]:
df = df[['Name', 'M/F', 'Age', 'Country', 'City', 'Pace', '5K', '10K', '15K', '20K', '25K', '30K', '35K', '40K', 'Official Time', 'Year']]

#### CHANGING TYPE OF COLUMNS THAT ARE USING TIME VALUES

In [13]:
time_columns = ['Pace', '5K', '10K', '15K', '20K', '25K', '30K', '35K', '40K', 'Official Time']
df[time_columns] = df[time_columns].apply(lambda col: pd.to_timedelta(col, errors='coerce'))

#### CONVERTING MILES TO KILOMETERS IN 'Pace' COLUMN

In [15]:
df['Pace'] = df['Pace'] / 1.609344

## ADDING NEW COLUMNS

## 'RANK' COLUMN
MAKING NEW 'RANK' COLUMN THAT REPRESENT RANK OF EACH RUNNER BY OFFICIAL TIME

In [18]:
df = df.sort_values(by='Official Time').reset_index(drop=True)
df['Rank'] = pd.Series(df.index) + 1

## 'AGE GROUP' COLUMN
MAKING NEW 'AGE GROUP' COLUMN THAT REPRESENT AGE GROUP OF EACH RUNNER (e.g. 22–25)

In [20]:
age = df['Age']
age_max = age.max() # Max age of participants
age_min = age.min() # Min age of participants

### CREATING AGE GROUPS

In [22]:
age_edges = list(np.arange(age_min, 70, 4)) # Creating age group edges with maximum age of 69
age_edges.append(70) # Adding one more starting edge that equals 70
age_edges.append(age_max + 1) # Adding ending edge that equals maximum age +1
                                # Now last group will be 70+ 

age_groups = [[age_edges[i], age_edges[i+1]] for i in range(len(age_edges)-1)] # Creating age groups
print(f"Age groups: {age_groups}")

Age groups: [[18, 22], [22, 26], [26, 30], [30, 34], [34, 38], [38, 42], [42, 46], [46, 50], [50, 54], [54, 58], [58, 62], [62, 66], [66, 70], [70, 85]]


### CREATING COLUMN

In [24]:
labels = [f"{g[0]}–{g[1]-1}" for g in age_groups]
bins = [g[0] for g in age_groups] + [age_groups[-1][1]]

df['Age Group'] = pd.cut(age, bins=bins, labels=labels, right=False)

## 'CONTINENT' COLUMN
MAKING NEW 'CONTINENT' COLUMN THAT REPRESENT CONTINENT OF EACH COUNTRY OF RUNNER

### EVERY COUNTRY (IN THIS DF) LISTS THAT ARE NAMED BY THEIR CONTINENT

In [27]:
europe = ['UKR', 'NED', 'ITA', 'RUS', 'BEL', 'BLR', 'ESP', 'IRL', 'LAT', 'GBR', 'CRO', 'GER', 'SWE', 'SUI', 'SVK', 'DEN', 'NOR', 'FIN', 'POL', 'FRA', 'ISL', 'POR', 'AUT', 'LTU', 'CYP', 'CZE', 'SLO', 'EST', 'TUR', 'LUX', 'ROU', 'LIE', 'GRE', 'SRB', 'HUN', 'MLT', 'AND', 'SMR', 'BUL', 'ALB']
north_america = ['USA', 'CAN', 'MEX', 'GUA', 'CRC', 'DOM', 'PAN', 'BER', 'HON', 'CAY', 'ESA', 'JAM', 'VGB', 'NCA', 'BAR', 'GRN', 'BAH', 'TCA']
south_america = ['AHO' ,'BRA', 'ECU', 'COL', 'CHI', 'PER', 'ARG', 'URU', 'VEN', 'FLK', 'TRI', 'PAR']
asia = ['PAK', 'QAT' ,'MGL' ,'KUW', 'JPN', 'BRN', 'HKG', 'MAS', 'CHN', 'PHI', 'SIN', 'TWN', 'KOR', 'TPE', 'IND', 'UAE', 'VIE', 'ISR', 'THA', 'KSA', 'INA', 'JOR', 'OMA']
africa = ['NGR' ,'ETH', 'KEN', 'RSA', 'ZIM', 'UGA', 'BDI', 'EGY', 'MAR', 'ALG']
oceania = ['NZL', 'AUS']

### DICTIONARY WITH COUNTRY = CONTINENT

In [29]:
continent_map = {}

for country in europe:
    continent_map[country] = 'Europe'
    
for country in asia:
    continent_map[country] = 'Asia'
    
for country in africa:
    continent_map[country] = 'Africa'
    
for country in north_america:
    continent_map[country] = 'North America'
    
for country in south_america:
    continent_map[country] = 'South America'
    
for country in oceania:
    continent_map[country] = 'Oceania'

### NEW COLUMN

In [31]:
df['Continent'] = df['Country'].map(continent_map)

## CHANGING COLUMNS ORDER

In [33]:
df = df[['Name', 'M/F', 'Age', 'Age Group', 'Country', 'Continent', 'City', 'Pace', '5K', '10K', '15K', '20K', '25K', '30K', '35K', '40K', 'Official Time', 'Rank', 'Year']]

## CHECKING IF ALL GOOD

In [35]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 79638 entries, 0 to 79637
Data columns (total 19 columns):
 #   Column         Non-Null Count  Dtype          
---  ------         --------------  -----          
 0   Name           79638 non-null  object         
 1   M/F            79638 non-null  object         
 2   Age            79638 non-null  int64          
 3   Age Group      79638 non-null  category       
 4   Country        79638 non-null  object         
 5   Continent      79638 non-null  object         
 6   City           79637 non-null  object         
 7   Pace           79638 non-null  timedelta64[ns]
 8   5K             79409 non-null  timedelta64[ns]
 9   10K            79524 non-null  timedelta64[ns]
 10  15K            79587 non-null  timedelta64[ns]
 11  20K            79553 non-null  timedelta64[ns]
 12  25K            79557 non-null  timedelta64[ns]
 13  30K            79550 non-null  timedelta64[ns]
 14  35K            79552 non-null  timedelta64[ns]
 15  40

In [36]:
df.head()

,Name,M/F,Age,Age Group,Country,Continent,City,Pace,5K,10K,15K,20K,25K,30K,35K,40K,Official Time,Rank,Year
0,"Desisa, Lelisa",M,25,22–25,ETH,Africa,Ambo,0 days 00:03:03.925872902,0 days 00:14:43,0 days 00:29:43,0 days 00:44:57,0 days 01:00:29,0 days 01:16:07,0 days 01:32:00,0 days 01:47:59,0 days 02:02:39,0 days 02:09:17,1,2015
1,"Kirui, Geoffrey",M,24,22–25,KEN,Africa,Keringet,0 days 00:03:04.547244094,0 days 00:15:25,0 days 00:30:28,0 days 00:45:44,0 days 01:01:15,0 days 01:16:59,0 days 01:33:01,0 days 01:48:19,0 days 02:02:53,0 days 02:09:37,2,2017
2,"Tsegay, Yemane Adhane",M,30,30–33,ETH,Africa,Addis Ababa,0 days 00:03:05.168615286,0 days 00:14:43,0 days 00:29:43,0 days 00:44:58,0 days 01:00:28,0 days 01:16:07,0 days 01:31:59,0 days 01:47:59,0 days 02:02:42,0 days 02:09:48,3,2015
3,"Rupp, Galen",M,30,30–33,USA,North America,Portland,0 days 00:03:05.168615286,0 days 00:15:24,0 days 00:30:27,0 days 00:45:44,0 days 01:01:15,0 days 01:16:59,0 days 01:33:01,0 days 01:48:19,0 days 02:03:14,0 days 02:09:58,4,2017
4,"Chebet, Wilson",M,29,26–29,KEN,Africa,Marakwet,0 days 00:03:05.789986478,0 days 00:14:43,0 days 00:29:43,0 days 00:44:57,0 days 01:00:29,0 days 01:16:07,0 days 01:32:00,0 days 01:47:59,0 days 02:03:01,0 days 02:10:22,5,2015


## IMPORTING CLEAN DATASET TO NEW FILE

In [38]:
df.to_csv('boston_clean.csv', index=False)